In [13]:
import os 
from  dotenv  import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from  langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_groq import ChatGroq

load_dotenv()




True

In [14]:
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key = groq_api_key , model_name = "Llama3-8b-8192",temperature=0.5)

In [4]:
loader =  TextLoader("medium.txt")
document = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50 )
texts  = text_splitter.split_documents(document)

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()


c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\s\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
db = FAISS.from_documents(texts,embeddings)

In [35]:
db.save_local("faiss_index")

In [28]:
query = " What is Langgraph. Please provide example code?"
# chain = PromptTemplate.from_template(template=query)| llm

# result = chain.invoke(input={})

# print(result.content)


In [18]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [19]:
combine_docs_chain = create_stuff_documents_chain(llm,retrieval_qa_chat_prompt)

In [20]:
retrieval_chain = create_retrieval_chain(
    retriever= db.as_retriever() , combine_docs_chain=combine_docs_chain
)

In [29]:
result = retrieval_chain.invoke(input={'input':query})

In [34]:
result['answer']

'Based on the context, LangGraph appears to be a library or framework for creating and working with graphs. It seems to be a part of the LangChain project, and allows users to define a graph and then compile it into a "runnable" that exposes certain methods.\n\nUnfortunately, the provided context does not include example code for LangGraph. However, it does mention the `graph` module and the `END` constant, as well as the `Edges` class.\n\nHere is a hypothetical example of how LangGraph might be used:\n```\nfrom langgraph.graph import Graph, END\n\n# Define a graph\ngraph = Graph()\n\n# Add nodes to the graph\nnode1 = graph.add_node("Node 1")\nnode2 = graph.add_node("Node 2")\nnode3 = graph.add_node("Node 3")\n\n# Add edges to the graph\nedge1 = graph.add_edge(node1, node2, type="undirected")\nedge2 = graph.add_edge(node2, node3, type="directed")\n\n# Compile the graph into a runnable\nrunnable = graph.compile()\n\n# Use the runnable to perform graph operations\nresult = runnable.get_s